In [17]:
import pandas as pd
from scipy.spatial.distance import euclidean
from scipy.stats import zscore

# Load the dataset
file_path = 'QVI_data.csv'  
data = pd.read_csv(file_path)

# Convert DATE column to datetime format
data['DATE'] = pd.to_datetime(data['DATE'])

# Extract year and month from DATE for aggregation
data['YEAR_MONTH'] = data['DATE'].dt.to_period('M')

# Aggregate data at a monthly level
monthly_data = data.groupby(['YEAR_MONTH', 'STORE_NBR']).agg({
    'TOT_SALES': 'sum',
    'LYLTY_CARD_NBR': 'nunique',
    'TXN_ID': 'count'
}).reset_index()

# Rename columns for clarity
monthly_data.rename(columns={
    'TOT_SALES': 'total_sales',
    'LYLTY_CARD_NBR': 'total_customers',
    'TXN_ID': 'total_transactions'
}, inplace=True)

# Calculate average number of transactions per customer
monthly_data['avg_transactions_per_customer'] = monthly_data['total_transactions'] / monthly_data['total_customers']

# Define the pre-trial period (use a smaller subset of the data)
pre_trial_period_start = '2018-08'
pre_trial_period_end = '2019-01'

# Filter to the pre-trial period
pre_trial_period = monthly_data[(monthly_data['YEAR_MONTH'] >= pre_trial_period_start) & 
                                (monthly_data['YEAR_MONTH'] <= pre_trial_period_end)]

# Check the number of months each store has data for in the pre-trial period
store_month_counts = pre_trial_period.groupby('STORE_NBR')['YEAR_MONTH'].nunique()
print("Distribution of months with data per store:\n", store_month_counts.describe())

# Define the minimum number of months a store should have data for to be considered
min_months_threshold = 5  
full_obs_stores = store_month_counts[store_month_counts >= min_months_threshold].index.tolist()

# Filter pre-trial period data to include only stores with data for at least min_months_threshold months
pre_trial_period = pre_trial_period[pre_trial_period['STORE_NBR'].isin(full_obs_stores)]

# Display the first few rows of the pre-trial period data
print(pre_trial_period.head())

# Calculate similarity using Euclidean distance
def calculate_similarity(trial_store, metric):
    distances = {}
    trial_data = pre_trial_period[pre_trial_period['STORE_NBR'] == trial_store].set_index('YEAR_MONTH')[metric]
    
    for store in pre_trial_period['STORE_NBR'].unique():
        if store != trial_store:
            store_data = pre_trial_period[pre_trial_period['STORE_NBR'] == store].set_index('YEAR_MONTH')[metric]
            
            # Ensure there is sufficient data and align months
            common_months = trial_data.index.intersection(store_data.index)
            if len(common_months) > 1:
                aligned_trial_data = trial_data.loc[common_months]
                aligned_store_data = store_data.loc[common_months]
                distance = euclidean(aligned_trial_data, aligned_store_data)
                distances[store] = distance
    
    return distances

# Define a function to find the best control store based on multiple metrics
def find_best_control_store(trial_store):
    metrics = ['total_sales', 'total_customers', 'avg_transactions_per_customer']
    store_scores = {store: 0 for store in pre_trial_period['STORE_NBR'].unique() if store != trial_store}
    
    for metric in metrics:
        distances = calculate_similarity(trial_store, metric)
        sorted_stores = sorted(distances, key=distances.get)
        
        for rank, store in enumerate(sorted_stores):
            store_scores[store] += rank
    
    best_control_store = int(min(store_scores, key=store_scores.get, default=None))
    return best_control_store

# Find best control stores for each trial store
trial_stores = [77, 86, 88]
control_stores = {trial_store: find_best_control_store(trial_store) for trial_store in trial_stores}

# Display the selected control stores
print(control_stores)


Distribution of months with data per store:
 count    268.000000
mean       5.902985
std        0.634459
min        1.000000
25%        6.000000
50%        6.000000
75%        6.000000
max        6.000000
Name: YEAR_MONTH, dtype: float64
    YEAR_MONTH  STORE_NBR  total_sales  total_customers  total_transactions  \
266    2018-08          1       176.10               42                  43   
267    2018-08          2       193.80               39                  43   
268    2018-08          3      1079.75              112                 134   
269    2018-08          4      1259.50              123                 151   
270    2018-08          5       745.10               97                 112   

     avg_transactions_per_customer  
266                       1.023810  
267                       1.102564  
268                       1.196429  
269                       1.227642  
270                       1.154639  
{77: 111, 86: 101, 88: 237}
